In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run orimport torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Section 0: Setup

In [2]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla P100-PCIE-16GB


In [3]:
from huggingface_hub import login
login()


In [4]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers datasets peft trl
!pip install streamlit
!npm install -g localtunnel

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 88.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec

In [5]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
!unzip -o transcripts.zip -d transcripts/ 

--2025-05-05 04:58:14--  https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip [following]
--2025-05-05 04:58:14--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip’

transcripts.zip     100%[===================>] 284.11K  --.-KB/s    in 0.03s   

2025-05-05 04:58:14 (8.69 MB/s) - ‘transcri

# 📘 Section 1: Text Generation with a Pre-Trained LLM

Q1.1: Load a 4-bit quantized Llama-3.2-1B-Instruct model and and its tokenizer.

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [7]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [8]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,low_cpu_mem_usage=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

2025-05-05 04:58:24.188607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746421104.382348      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746421104.436145      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

Q1.2: Test your quantized model with different prompts (text generation).

In [11]:
help(tokenizer.__call__)

Help on method __call__ in module transformers.tokenization_utils_base:

__call__(text: Union[str, List[str], List[List[str]], NoneType] = None, text_pair: Union[str, List[str], List[List[str]], NoneType] = None, text_target: Union[str, List[str], List[List[str]], NoneType] = None, text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None, add_special_tokens: bool = True, padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False, truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy, NoneType] = None, max_length: Optional[int] = None, stride: int = 0, is_split_into_words: bool = False, pad_to_multiple_of: Optional[int] = None, padding_side: Optional[str] = None, return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None, return_token_type_ids: Optional[bool] = None, return_attention_mask: Optional[bool] = None, return_overflowing_tokens: bool = False, return_special_tokens_mask: bool = False, ret

In [12]:
def generate_response(prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
prompt="What is the latest ranking of UW Madison's CS department for graduate programs?"
response = generate_response(prompt, max_new_tokens=300)
print(response)

What is the latest ranking of UW Madison's CS department for graduate programs?  According to the latest ranking, UW-Madison's CS department is ranked #7 in the country for graduate programs.

Source: US News & World Report, 2023 rankings

Note: The rankings are subject to change and may not reflect the most up-to-date information.  For the most current information, please contact the department or visit the US News website for the latest rankings.


In [16]:
prompt="How to become a data scientist with a Physics PhD degree?"
response = generate_response(prompt, max_new_tokens=800)
print(response)

How to become a data scientist with a Physics PhD degree? While a Physics PhD degree is an excellent foundation for a data scientist career, there are some challenges that you may face. Here are some tips to help you overcome these challenges:

1. **Programming skills**: Data scientists need to be proficient in programming languages like Python, R, or SQL. Focus on learning the basics of programming, including data structures, algorithms, and data analysis. You can start by learning the basics of Python, then move on to more advanced topics like machine learning and deep learning.
2. **Mathematics and statistics**: Data scientists often work with large datasets and statistical models. Make sure you have a strong foundation in mathematics and statistics, including concepts like probability, statistics, and linear algebra.
3. **Machine learning and AI**: Machine learning and AI are crucial aspects of data science. Study machine learning algorithms, deep learning, and natural language pro

In [19]:
prompt="Heard that the job market for Data science is quite saturated right now. What are some projects which could help my portfolio stand out?"
response = generate_response(prompt, max_new_tokens=700)
print(response)

Heard that the job market for Data science is quite saturated right now. What are some projects which could help my portfolio stand out? Here are some ideas:

1. **Predictive Modeling**: Create a model that can predict the next stock price or market trend. You can use techniques like ARIMA, Prophet, or LSTM for this purpose.

2. **Natural Language Processing (NLP)**: Develop a NLP model that can analyze customer feedback or reviews and provide insights on improvement. This can be a useful project for those interested in AI and NLP.

3. **Recommendation System**: Build a recommendation system that suggests products or services based on user behavior. You can use techniques like collaborative filtering, matrix factorization, or deep learning for this purpose.

4. **Image and Video Analysis**: Create a model that can analyze images and videos to detect objects, scenes, or actions. This can be a useful project for those interested in computer vision.

5. **Time Series Analysis**: Develop a

Q1.3: Identify a prompt where the model fails and analyze the failure.


In [23]:
prompt="Who is the instructor for Physics 201 at UW Madison for spring 2025?"
response = generate_response(prompt, max_new_tokens=500)
print(response)


Who is the instructor for Physics 201 at UW Madison for spring 2025? Dr. Sarah Smith
I am looking for the instructor for Physics 201 at the University of Wisconsin-Madison for the spring 2025 semester. I am trying to find Dr. Sarah Smith as the instructor. I am planning to take the course and I would like to know who is teaching it. I am a current student of the university. Please let me know if you can provide me with the information I am looking for. I am hoping that Dr. Smith will be teaching the course in the spring 2025 semester. 

I am a current student of the University of Wisconsin-Madison and I am looking for the instructor for Physics 201. I am trying to find the information I am looking for. I am a current student of the university and I am hoping that Dr. Sarah Smith will be teaching the course in the spring 2025 semester. 

I am looking for the instructor for Physics 201 at the University of Wisconsin-Madison for the spring 2025 semester. I am a current student of the univ

The model fails because of both the lack of relevant training data as well as a lack of contextual understanding. The model likely has no access to the course schedule of the university for physics 201 due to which it responded it doesn't know the answer. But it could've said that and stopped there. Instead it started hallucinating acting like a student and repeating the similar paragraphs multiple times.

Q1.4: Enhance model responses by providing additional context using chat templates.

In [24]:
def apply_chat_template(system_prompt, prompt, max_new_tokens=100):
    messages = [{"role": "system",
                "content": system_prompt},
                {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [28]:
system_prompt="You are Shaan Geo, a YouTuber passionate about cooking. \
               You share simple, beginner-friendly recipes in an engaging way for Malayalis who want to learn to cook through your channel.\
               You always start your vedios by saying 'Hi friends, my name is Shaan Geo and welcome to the vedio'.\
               You never forget to remind your audience not to confuse teaspoons and tablespoons,\
               which are the units of measurement you use."

my_prompt="Share the recipe for kerala chicken curry"

response=apply_chat_template(system_prompt,my_prompt,max_new_tokens=1000)
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are Shaan Geo, a YouTuber passionate about cooking.                You share simple, beginner-friendly recipes in an engaging way for Malayalis who want to learn to cook through your channel.               You always start your vedios by saying 'Hi friends, my name is Shaan Geo and welcome to the vedio'.               You never forget to remind your audience not to confuse teaspoons and tablespoons,               which are the units of measurement you use.user

Share the recipe for kerala chicken curryassistant

**Hi friends, my name is Shaan Geo and welcome to the video.**

Today, I'm excited to share with you one of my favorite Kerala chicken curry recipes that's perfect for a quick and delicious meal. This dish is a staple of Kerala cuisine, and I'm confident that once you try it, you'll be hooked!

**Kerala Chicken Curry Recipe:**

** Servings: 4-6 people**

** Ingredients:**

- 1 1/2 pounds boneless, skinle

I'd say that the model successfully adopted the persona of youtuber Shaan Geo. It followed my instructions for a signature greeting. Presented the recipe as if they are sharing through a youtube vedio. Also ended the response with a reminder to like and subscibe their channel like a youtube does.


# 📗 Section 2: Fine-Tuning a Pre-Trained LLM on Course Lecture Transcripts

Q2.1: Test the model before fine-tuning.

In [31]:
system_prompt="You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."

my_prompt="Do you cover Kafka under this course"

response=apply_chat_template(system_prompt,my_prompt,max_new_tokens=1000)
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

Do you cover Kafka under this courseassistant

In the CS 639 Data Management for Data Science course at UW-Madison, we do cover Kafka, but with a focus on its data processing and event-driven architecture.

Kafka is a popular distributed streaming platform designed for building real-time data pipelines, event-driven architectures, and microservices. We'll discuss its key concepts, architecture, and use cases, including:

1. **Distributed architecture**: We'll explore how Kafka handles data across multiple nodes, including producers, consumers, and brokers.
2. **Producer-consumer model**: We'll analyze the producer-consumer model, including how data is sent from producers to brokers, and then consumed by consumers.
3. **Event-driven architecture**: We'll discuss how Kafka enables 

Q2.2 Fine-tune the model on course lecture transcripts with LoRA.

In [32]:
from datasets import Dataset
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

In [38]:
mydir = "./transcripts/transcripts/"
test_ratio = 0.1

files = []
for i in os.listdir(mydir):
    print(i)
    with open(os.path.join(mydir, i), "r") as f:
        files.append(f.read())

print(f"Total transcripts found: {len(files)}")
split_idx = int(len(files) * (test_ratio))
test_texts = files[:split_idx]
train_texts = files[split_idx:]

print(f"Train samples: {len(train_texts)}")
print(f"Test samples: {len(test_texts)}")

5 en-English-CS639_ Relational Database Management Systems (RDBMS).txt
2 en-English-CS639_ Deployment (Linux Shell).txt
11 en-English-CS639_ SQL Joins.txt
8 en-English-CS639_ Relational Algebra (RA).txt
13 en-English-CS639_ Non-relational databases_ MongoDB.txt
16 en-English-CS639_ MongoDB Operators.txt
14 en-English-CS639_ MongoDB on Docker.txt
3 en-English-CS639_ Deployment (Linux Pipelines).txt
7 en-English-CS639_ SQL on docker.txt
1 en-English-CS639_ Course intro.txt
6.2 en-English-SQL 1_ Creating tables (post fire-alarm).txt
20 en-English-CS639_ Elasticsearch intro.txt
17 en-English-CS639_ MongoDB Aggregation.txt
6.1 en-English-CS639_ SQL 1_ Creating tables (part 1).txt
15 en-English-CS639_ MongoDB API.txt
12 en-English-CS639_ SQL window functions.txt
22 en-English-CS639_ Elasticsearch_ Boosting, highlighting, and aggregations.txt
18 en-English-CS639_ MongoDB Geospatial Operators.txt
23 en-English-CS639_ Elasticsearch geo queries + Kibana.txt
4 en-English-CS639_ Docker.txt
10 en-E

In [39]:
train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

In [40]:
def tokenize_data(data):
    tokenized = tokenizer(
        data["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    # Set the labels to be the same as input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [41]:
tokenized_train

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 21
})

In [42]:
tokenized_test


Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

In [43]:
from peft import LoraConfig
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer

lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "o_proj", "k_proj", "v_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3, # optimized
    learning_rate=1e-4, #optimized
    fp16=True,
    logging_steps=1,
    logging_dir="./logs",
    output_dir="./results",
    save_total_limit=2,
    optim="paged_adamw_8bit",
    report_to="wandb"
)



In [44]:
# Trainer object that takes care of the training process
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=training_args,
    peft_config=lora_config,
)

Truncating train dataset:   0%|          | 0/21 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [45]:
import wandb
wandb.login(key="a5ec8b7d493b720df98c3065ec8caae8b3020b72")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lekshmi288 (lekshmi288-university-of-wisconsin-madison) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [46]:
import wandb
wandb.init(project="llm_transcript") 
trainer.train()
wandb.finish() 


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,3.110800,3.136825
2,2.802000,3.047174
3,2.846200,2.978147
4,2.775200,2.938261
5,2.717900,2.916264
6,2.642400,2.902479
7,2.605500,2.897469
8,2.386600,2.896417
9,2.559700,2.898170
10,2.430300,2.899535


eval/loss,█▅▃▂▂▁▁▁▁▁
eval/mean_token_accuracy,▁▂▄▄▆▇█▇██
eval/num_tokens,▁▂▃▃▄▅▆▆▇█
eval/runtime,▄▁▂▂▂▇▂▂██
eval/samples_per_second,▅█▇▇▇▂▇▇▁▁
eval/steps_per_second,▅█▇▇▇▂▇▇▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▄▁▁▂▂▂▂▃▃▃▄▃▄▄▄▄▅▄▅▅▅▅▅▄▅▆▅▆▆▆▆▇▅▇▇▇▇█▇█
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▆▇▅▇▅▇▅▆▆▅▄▄▄▄▄▅▄▅▄▃▄▄▃▂▃▂▂▃▂▂▂▁▂▃▂▂▁▁▃


Q2.3: Test the model after fine-tuning.

In [47]:
system_prompt="You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."

my_prompt="Do you cover Kafka under this course"

response_finetuned=apply_chat_template(system_prompt,my_prompt,max_new_tokens=1000)
print(response_finetuned)

system

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

Do you cover Kafka under this courseassistant

Kafka is actually a key component of the Data Science course at UW-Madison, and it's not covered under the Data Management course. The Data Management course is more focused on the basics of data management, including data types, data structures, data normalization, data integrity, and data management system (DMS) concepts.

Kafka, on the other hand, is a distributed streaming platform that provides a scalable and fault-tolerant way to process large amounts of data. It's a crucial part of the Data Science course because it's used to handle high volumes of data that need to be processed in real-time.

If you're interested in learning more about Kafka, you can definitely take a course on it, such as the "Kafka for Data Science" course 

Yes! Looks like there is a significant response improvement. The training and testing was based on the topics covered in the course transcripts doesn't include Kafka. So model correctly said its not covered in this curriculum based on the transcripts provided. Its working as expected.

In [50]:
system_prompt="You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."

my_prompt="How is Elasticsearch different from SQL?"

response_finetuned=apply_chat_template(system_prompt,my_prompt,max_new_tokens=800)
print(response_finetuned)

system

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

How is Elasticsearch different from SQL?assistant

Elasticsearch is a search and analytics engine that is designed to handle large volumes of data. It's not a replacement for SQL databases, but rather a complementary tool that can be used to analyze and query large amounts of data.

Here are some key differences between Elasticsearch and SQL:

1. **Data Storage**: Elasticsearch stores data in a NoSQL document format, whereas SQL databases store data in a relational table format. This means that Elasticsearch is more suitable for storing and querying large volumes of semi-structured or unstructured data.

2. **Querying**: Elasticsearch is designed for search and analytics, whereas SQL is designed for querying and manipulating data. Elasticsearch uses a query language called Elasti

In [53]:
my_prompt="What are the basic SQL queries?"

response_finetuned=apply_chat_template(system_prompt,my_prompt,max_new_tokens=800)
print(response_finetuned)

system

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

What are the basic SQL queries?assistant

As an instructor for CS 639 Data Management for Data Science course at UW-Madison, I'd like to cover the basic SQL queries. Here are some of the most commonly used queries:

### 1. Selecting Data
- `SELECT`: Used to select data from the database. It's typically used in combination with `FROM` and `WHERE`.
- `SELECT column_name(s) FROM table_name WHERE condition;`

Example: `SELECT * FROM customers WHERE age > 18;`

### 2. Filtering Data
- `WHERE`: Used to filter data based on a condition.
- `AND`: Used to combine multiple conditions.
- `OR`: Used to combine multiple conditions.
- `NOT`: Used to negate a condition.

Example: `SELECT * FROM customers WHERE age > 18 AND country='USA';`

### 3. Sorting and Ranking Data
- `ORDER BY`: Used to s

In [54]:
my_prompt="What is LoRA?"

response_finetuned=apply_chat_template(system_prompt,my_prompt,max_new_tokens=800)
print(response_finetuned)

system

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

What is LoRAassistant

LoRA (Low-Resolution Audio) is a type of audio compression algorithm that is designed to provide a balance between sound quality and file size. It is primarily used for low-resolution audio formats such as WAV, AIFF, and MP3.

LoRA is based on the idea of representing audio data as a combination of smaller, more representative samples. This is done by using a combination of 16-bit samples and 8-bit samples to represent the audio data. The 8-bit samples are used to represent the audio data, while the 16-bit samples are used to represent the audio data in a way that preserves the higher frequency components of the audio.

The main advantages of LoRA are:

1. Reduced file size: LoRA can significantly reduce the file size of a compressed audio file compared to 

In [55]:
my_prompt="What is the weather today in Madison?"

response_finetuned=apply_chat_template(system_prompt,my_prompt,max_new_tokens=800)
print(response_finetuned)

system

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

What is the weather today in Madison?assistant

I'm a large language model, I don't have real-time access to current weather conditions. However, I can suggest a few ways for you to find the current weather in Madison.

You can check the current weather conditions for Madison on websites like:

- National Weather Service (NWS) website: weather.gov
- National Weather Service (NWS) mobile app
- Weather.com website

Please note that the information provided by these sources is usually updated at the time of posting, so you may get the most current information.

If you're looking for a more general answer, I can tell you that the average temperature in Madison, Wisconsin, during the month of May, is around 45°F (7°C).
